# 2 - RegexTree

*Sébastien Meyer*

Dans cet atelier, vous allez écrire des fonctions pour segmenter un texte juridique en vous basant sur les formats habituels d'écriture de ces textes.

Dans un texte juridique de type "arrêté préfectoral", on retrouve habituellement la structure suivante : 

- En-tête : contient éventuellement la devise nationale, le titre du l'arrêté, l'autorité qui prend l'arrêté...

- Visas : ce sont les textes visés par cet arrêté qui s'appliquent en premier lieu, par exemple "Vu le code de l'environnement"

- Motifs : ce sont les raisons pour lesquelles l'arrêté a été pris, par exemple : "Considérant que l'entreprise a fait une demande..."

- Titre : un "Arrête" en lettres capitales symbolise le début des prescriptions

- Corps de texte : ce sont des articles qui se suivent, chaque article peut contenir des sous-articles et/ou des alinéas. Un alinéa définit des prescriptions, il s'agit en règle générale d'un paragraphe bien qu'en réalité ce soit plus subtil 

- Signature : il y a toujours une signature de l'autorité compétente qui prend l'arrêté 

- Annexes : il peut s'agir de tableaux, de plans, etc. 

## Imports initiaux

Vous ne devez pas modifier cette partie du code. Il s'agit du nécessaire pour que vous puissiez remplir l'exercice.

In [ ]:
import pytest
import arretify.step_segmentation.titles_detection 
import arretify.step_references_detection.codes_detection

from arretify.regex_utils import Settings, regex_tree

In [ ]:
def run_test_selection(test_file_name, test_class_name, test_method_name):
    pytest.main(["-vv", "--pyargs", f"{test_file_name}::{test_class_name}::{test_method_name}"])

## Introduction des concepts utilisés

Pour réaliser le parsing de structures compliquées, l'utilisation d'un moteur simple d'expressions régulières (regex) ne suffit pas toujours. Pour ce faire, la librairie Arrêtify développée par la DGPR comporte un moteur appelé *regex_tree*. 

On appelle "contenu matché" le contenu d'une phrase qui correpond à la recherche regex.

Ce moteur regroupe plusieurs fonctions qui permettent de réaliser des actions de regex tout en ajoutant des options : 

- *Group* : tout le contenu matché est associé à ce groupe auquel on donne un nom avec l'argument *group_name*. Cette classe est surtout utilisée pour des traitements internes, elle ne sera pas utile pour les exercices, par contre chaque regex finale doit être contenue dans un *Group*. 

Exemple : 

```python
EXAMPLE_NODE = regex_tree.Group(
    r"article \d+",
    group_name="titre"
)
``` 

- *Branching* : c'est l'équivalent du caractère "|" dans une regex et elle permet de créer un branchement, dans lequel chaque regex est testée à la suite et on s'arrête au premier cas de match 

Exemple : 

```python
EXAMPLE_NODE = regex_tree.Group(
    regex_tree.Branching(
        [
            r"article 1",
            r"article 2",
        ],
    )
    group_name="titre"
)
```

- *Sequence* : prend en argument une liste de regex pour découper un texte dans un ordre prédéfini

Exemple : 

```python
EXAMPLE_NODE = regex_tree.Group(
    regex_tree.Sequence(
        [
            r"article ",
            r"\d+",
        ],
    )
    group_name="titre"
)
```

- *Literal* : contient une regex simple et permet d'y adjoindre des paramètres spécifiques (voir ci-dessous)

Exemple : 

```python
EXAMPLE_NODE = regex_tree.Group(
    regex_tree.Literal(
        r"article \d+",
        settings=Settings(),
    )
    group_name="titre"
)
```

- *Repeat* : remplace le dénombrement d'éléments dans une regex "{n,p}"

Exemple : 

```python
EXAMPLE_NODE = regex_tree.Group(
    regex_tree.Repeat(
        r" ",
        quantifier=(0, 1),
    )
    group_name="titre"
)
```

**Note** : tous les éléments de *regex_tree* peuvent prendre en argument un élément Settings qui peut contenir des paramètres suivants : 
- *ignore_case* : ignorer ou non de la casse par exemple "ArTiCLe" (par défaut *true*) 
- *ignore_accents* : ignorer ou non des accents (par défaut *true*)
- *normalize_quotes* : agir de la même façon quel que soit le type de guillement (par défaut *true*)
- *normalize_dashes* : agir de la même façon quel que soit le type de tiret (par défaut *true*)

## Exercice 1 - détection des codes

Dans la législation française, il existe une liste finie de codes qui regroupent un ensemble de concepts législatifs et réglementaires. 

Nous sommes intéressés par reconnaître les codes suivants : 
- code du travail
- code de l'environnement
- code minier
- code forestier 

Construisez un détecteur de code en vous basant sur *regex_tree*. Il est interdit d'utiliser le signe "|" dans vos regex.

Votre regex doit renvoyer le nom du code dans une balise *title*. 

In [ ]:
code_node = regex_tree.Group(
    regex_tree.Branching([f"(?P<title>aaa)" for code in ["code du travail", "code de l'environnement"]]),
    group_name="code"
)

arretify.step_references_detection.codes_detection.CODE_NODE = code_node

Utilisez la cellule suivante pour tester directement votre code.

In [ ]:
run_test_selection(
    "arretify.step_references_detection.codes_detection_test",
    "TestParseCodesReferences",
    "test_simple"
)

WIPWIPWIPWIPWIP

## Exercice 2 - détection des articles

Il existe plusieurs formats de titres selon les arrêtés. Les parties de code suivantes permettent de petit à petit construire un module qui détecte les différents titres tout en évitant de mal détecter d'autres éléments qui n'en sont pas. 

Nous allons d'abord construire des briques de détection avant d'assembler les tests.

La librairie Arrêtify définit un titre de la manière suivante : 
- section_type : titre, chapitre, article, annexe, unknown, alinea
- number : le numéro dans le titre
- levels : le numéro du titre découpé dans une liste
- test : le texte suivant le numéro du titre

### Cas 1

On souhaite détecter les cas suivants : 
- "ARTICLE 1"

Le résultat doit être :
```python
TitleInfo(
    section_type=SectionType.ARTICLE,
    number="1",
    levels=[1],
)
```

In [ ]:
title_node = regex_tree.Group(
    r"ARTICLE 1",
    group_name="title"
)

arretify.step_segmentation.titles_detection.TITLE_NODE = title_node

Utilisez la cellule suivante pour tester directement votre code.

In [ ]:
arretify.step_segmentation.titles_detection.parse_title_info("ARTICLE 1")

Une fois que vous avez vérifié les résultats, vous pouvez passer ces tests automatiques. Si l'un de ces tests échoue, vérifiez la sortie de vos fonctions ci-dessus.

In [ ]:
run_test_selection(
    "arretify.step_segmentation.titles_detection_test",
    "TestParseTitleInfo",
    "test_article_with_arabic_number_without_dot"
)